PILAR ARIAS, YEDRA MARTÍN, PATRICIA MEDIAVILLA, AMAYA SARASA

<GRUPO 9>

#Práctica 1

##Introducción

Esta es la primera práctica del segundo laboratorio. Dada una base de datos en formato .csv, se nos pide contestar a una serie de cuestiones. Para ello, lo priemro que debemos hacer es cargar y leer la base de datos:

In [ ]:
#Importar librerías
#!apt install mlxtend
import numpy as np #numpy
import pandas as pd #pandas
import sklearn as sk #scikit-learn
import matplotlib.pyplot as plt #gráfica Matplotlib
#from mlxtend.frequent_patterns import apriori
#from apyori import apriori #apriori
from mlxtend.preprocessing import TransactionEncoder
#from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori, association_rules
#from mlxtend.frequent_patterns import frequent_patterns
#from mlxtend import frequent_patterns
#from mlxtend.frequent_patterns import frequent_patterns

#Importamos la base de datos
df = pd.read_csv('BlackFriday.csv',sep=',')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51032 entries, 0 to 51031
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     51032 non-null  int64  
 1   Product_ID                  51032 non-null  object 
 2   Gender                      51031 non-null  object 
 3   Age                         51031 non-null  object 
 4   Occupation                  51031 non-null  float64
 5   City_Category               51031 non-null  object 
 6   Stay_In_Current_City_Years  51031 non-null  object 
 7   Marital_Status              51031 non-null  float64
 8   Product_Category_1          51031 non-null  float64
 9   Product_Category_2          35294 non-null  float64
 10  Product_Category_3          15545 non-null  float64
dtypes: float64(5), int64(1), object(5)
memory usage: 4.3+ MB


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7.0,B,2,1.0,1.0,11.0,NaN
1,1000009,P00113442,M,26-35,17.0,C,0,0.0,3.0,5.0,NaN
2,1000010,P00288442,F,36-45,1.0,B,4+,1.0,5.0,14.0,NaN
3,1000010,P00145342,F,36-45,1.0,B,4+,1.0,4.0,9.0,NaN
4,1000011,P00053842,F,26-35,1.0,C,1,0.0,4.0,5.0,12.0


----------------

----------------

Subimos la base de datos, df.info nos aporta la infomación que contiene la base de datos, vemos el tipo, la cantidad de las variables con las que trabajamos... y .head vamos a ver que forma tiene la base de datos y las primeras filas de esta.

Según se ha estudiado en clase, encontramos dos métodos para afrontarlo. Nos decantaremos por CRISP-DM. Las siglas para: CRoss-Industry Standard Process for Data Mining. 

Es un modelo de proceso de minería de datos que se utiliza en la industria. Es un marco de trabajo que define un proceso estandarizado para llevar a cabo proyectos de minería de datos, desde la definición del problema hasta la implementación y evaluación del resultado. Los seis pasos que componen Crisp-DM son los siguientes:

- Comprensión de los negocios y definición del problema: comprender la naturaleza del problema y los objetivos de negocio.

- Comprensión de los datos: recopilación inicial, descripción y exploración  de datos.

- Preparación de los datos: Selección, limpieza, construcción, integración, formateo de los datos.

- Modelado: aplicar las técnicas de minería de datos a los dataset.

- Evaluación: determinar si los resultados son útiles a las necesidades del negocio.

- Despliegue: explotar utilidad de los modelos, integrándolos en las tareas de toma de decisiones de la organización -> Call to Action. 

##Comprensión de los negocios y definición del problema

La librería se usará de la siguiente manera:
1) Empezaremos obteniendo los itemsets frecuentes para k=1. En este punto necesitaremos obtener el
soporte de los itemset. Por lo tanto, se tendrá que usar un método que dado un itemset devuelva su
soporte.

2) Para k≥2.
a. Mostrar los itemset frecuentes candidatos y su soporte.

3) Repetir el proceso 2 hasta que no se generen nuevos itemsets frecuentes.

4) Mostrar todas las posibles reglas con la confianza de cada una de ellas.

5) Listar todas las reglas que sean de alta confianza.

6) Usar los siguientes métodos. Dado un antecesor, devolver todas las reglas que contengan a dicho
antecesor. Dado un umbral mínimo devolver todas las reglas que cumplan con dicha confianza.

7) Utiliza las representaciones gráficas que consideres adecuadas para representar las reglas obtenidas
y obtener conclusiones a partir de los datos.

A partir de lo anterior de pide:
- Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto.
- ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos 
- ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto)
consideradas? ¿Por qué? 
- Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes
obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y
min_threshold. Justifica los resultados. 
- Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de
alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos 

## Comprensión de los datos

La base de datos que utilizaremos consta de 233599 observaciones. Teniendo en ceunta que el trabajo que vamos a realizar, utilizanod el algoritmo apriori, queremos sacar regalas y con todos estos datos, lo que nos hace pensar es que vamos a encontrar muchísimas reglas a partir de estos datos.

La clave está en cuales son las que más nos interesan, basandonos en su confianza, y una vez hallado esto debemos tomar decisiones para que usando estas reglas vendamos el mayor número de producto, para aumentar nuestras gananzas.

Existen 11 tipos de atributos distintos: 

- User_ID: número para identificar al usuario.
- Product_ID: número para identificar el producto.
- Gender: Género del comprador.	
- Age: Edad del comprador.
- Occupation: A lo que se dedica el comprador, ya pasasdo a categórica.
- City_Category: Categoría de la ciudad donde reside el user.
- Stay_In_Current_City_Years: El tiempo que lleva en la ciudad.
- Marital_Status: Estado civil.
- Product_Category_1: La categoría principal del producto.
- Product_Category_2: Siguente categoría.
- Product_Category_3: Última categoría del producto.



##Preparación de los datos

In [ ]:
usernan = df['User_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable User_ID es:{usernan}")
productnan = df['Product_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_ID es:{productnan}")
gendernan = df['Gender'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Gender es:{gendernan}")
agenan = df['Age'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Age es:{agenan}")
ocupationan = df['Occupation'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Occupation es:{ocupationan}")
City_Categorynan = df['City_Category'].isna().sum()
print(f"La cantidad de NaN que existen en la variable City_Category es:{City_Categorynan}")
Stay_In_Current_City_Yearsnan = df['Stay_In_Current_City_Years'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:{Stay_In_Current_City_Yearsnan}")
Marital_Statusnan = df['Marital_Status'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Marital_Status es:{Marital_Statusnan}")
Product_Category_1nan = df['Product_Category_1'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_1 es:{Product_Category_1nan}")
Product_Category_2nan = df['Product_Category_2'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_2 es:{Product_Category_2nan}")
Product_Category_3nan = df['Product_Category_3'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_3 es:{Product_Category_3nan}")

La cantidad de NaN que existen en la variable User_ID es:0
La cantidad de NaN que existen en la variable Product_ID es:0
La cantidad de NaN que existen en la variable Gender es:1
La cantidad de NaN que existen en la variable Age es:1
La cantidad de NaN que existen en la variable Occupation es:1
La cantidad de NaN que existen en la variable City_Category es:1
La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:1
La cantidad de NaN que existen en la variable Marital_Status es:1
La cantidad de NaN que existen en la variable Product_Category_1 es:1
La cantidad de NaN que existen en la variable Product_Category_2 es:15738
La cantidad de NaN que existen en la variable Product_Category_3 es:35487


In [ ]:
# Rellenamos los valores na con 0, ya que no tiene categoría
df = df.fillna(0)

# Quitamos los duplicados
df = df.drop_duplicates()

##Modelado

In [ ]:
df_User_Product = (df
        .groupby(['User_ID', 'Product_ID'])['Product_Category_1']
        .sum()
        .apply(lambda x:x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
          )

In [ ]:
df_Gender = (df
            .groupby(['User_ID','Gender'])['Product_Category_1']
            .sum()
            .apply(lambda x: x>0)
            .unstack()
            .fillna(False)
            .reset_index()
            .set_index('User_ID')
)

In [ ]:
df_Age = (df
        .groupby(['User_ID','Age'])['Product_Category_1']
        .sum()
        .apply(lambda x: x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
)

In [ ]:
df_Occupation = (df
                .groupby(['User_ID','Occupation'])['Product_Category_1']
                .sum()
                .apply(lambda x: x>0)
                .unstack()
                .fillna(False)
                .reset_index()
                .set_index('User_ID')
)

In [ ]:
df_City = (df
          .groupby(['User_ID','City_Category'])['Product_Category_1']
          .sum()
          .apply(lambda x: x>0)
          .unstack()
          .fillna(False)
          .reset_index()
          .set_index('User_ID')
)

In [ ]:
df_Stay = (df
           .groupby(['User_ID','Stay_In_Current_City_Years'])['Product_Category_1']
           .sum()
           .apply(lambda x: x>0)
           .unstack()
           .fillna(False)
           .reset_index()
           .set_index('User_ID')
)

In [ ]:
df_Marital = (df
              .groupby(['User_ID','Marital_Status'])['Product_Category_1']
              .sum()
              .apply(lambda x: x>0)
              .unstack()
              .fillna(False)
              .reset_index()
              .set_index('User_ID')
)

In [ ]:
df_Product_Category_1 = (df
                         .groupby(['User_ID','Product_Category_1'])['Product_Category_2']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [ ]:
df_Product_Category_2 = (df
                         .groupby(['User_ID','Product_Category_2'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [ ]:
df_Product_Category_3 = (df
                         .groupby(['User_ID','Product_Category_3'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [ ]:
df_Product_Category_3.value_counts()

0.0   3.0    4.0    5.0    6.0    8.0    9.0    10.0   11.0   12.0   13.0   14.0   15.0   16.0   17.0   18.0 
True  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False    1241
                                                                                          True   False  False     265
                                                                            True   False  False  False  False     160
                                                                            False  False  False  True   False     156
                                                                                   True   False  False  False     146
                                                                                                                 ... 
                    True   False  False  False  False  False  False  True   True   True   True   False  False       1
                                                                

In [ ]:
df_Product_Category_1=df_Product_Category_1.set_axis(['0_PC0','1_PC1','2_PC1','3_PC1','4_PC1','5_PC1','6_PC1','7_PC1','8_PC1','9_PC1','10_PC1','11_PC1','12_PC1','13_PC1','14_PC1','15_PC1','16_PC1','17_PC1','18_PC1'], axis=1)

In [ ]:
df_Product_Category_2=df_Product_Category_2.set_axis(['0_PC2','2_PC2','3_PC2','4_PC2','5_PC2','6_PC2','7_PC2','8_PC2','9_PC2','10_PC2','11_PC2','12_PC2','13_PC2','14_PC2','15_PC2','16_PC2','17_PC2','18_PC2'], axis=1)

In [ ]:
df_Product_Category_3=df_Product_Category_3.set_axis(['0_PC3','3_PC3','4_PC3','5_PC3','6_PC3','8_PC3','9_PC3','10_PC3','11_PC3','12_PC3','13_PC3','14_PC3','15_PC3','16_PC3','17_PC3','18_PC3'],axis=1)

In [ ]:
df_Marital=df_Marital.set_axis(['0_Marital', '1_Marital'], axis=1)

In [ ]:
df_Occupation=df_Occupation.set_axis(['0_Occ','1_Occ', '2_Occ', '3_Occ', '4_Occ', '5_Occ', '6_Occ', '7_Occ', '8_Occ', '9_Occ', '10_Occ', '11_Occ', '12_Occ', '13_Occ', '14_Occ', '15_Occ', '16_Occ', '17_Occ', '18_Occ', '19_Occ', '20_Occ'], axis=1)

In [ ]:
df_Stay=df_Stay.set_axis(['0_Stay','1_Stay','2_Stay','3_Stay','4_Stay','5_Stay'],axis=1)

In [ ]:
data = pd.concat([df_User_Product, df_Occupation, df_City, df_Stay, df_Marital, df_Product_Category_1, df_Product_Category_2, df_Product_Category_3, df_Age, df_Gender], axis = 1)
data

,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,0-17,18-25,26-35,36-45,46-50,51-55,55+,0,F,M
User_ID,,,,,,,,,,,,,,,,,,,,,
1000001,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
1000002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
1000003,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1000004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1000005,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006036,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
1006037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
1006038,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [ ]:
print(data.shape)

(5535, 3210)


In [ ]:
#Esto es para k=1 // use_colnames = True, max_len = 3
#frq_items = apriori(df, min_support = 0.05, use_colnames = True, max_len = 3)

frequent_itemsets = apriori(data, min_support=0.05, use_colnames=True, max_len=3)
frequent_itemsets

,support,itemsets
0,0.118338,(0_Occ)
1,0.089431,(1_Occ)
2,0.125745,(4_Occ)
3,0.113640,(7_Occ)
4,0.064318,(12_Occ)
...,...,...
5863,0.100813,"(15_PC3, M, 26-35)"
5864,0.081301,"(M, 16_PC3, 17_PC3)"
5865,0.055465,"(M, 16_PC3, 18-25)"
5866,0.106956,"(M, 16_PC3, 26-35)"


In [ ]:
#Para k=1

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets1=frequent_itemsets[frequent_itemsets['length'] == 1]
frequent_itemsets1

,support,itemsets,length
0,0.118338,(0_Occ),1
1,0.089431,(1_Occ),1
2,0.125745,(4_Occ),1
3,0.113640,(7_Occ),1
4,0.064318,(12_Occ),1
...,...,...,...
58,0.089792,(46-50),1
59,0.082023,(51-55),1
60,0.060885,(55+),1
61,0.279133,(F),1


In [ ]:
#Para k=2

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets2=frequent_itemsets[frequent_itemsets['length'] == 2]
frequent_itemsets2

,support,itemsets,length
63,0.059801,"(C, 0_Occ)",2
64,0.069015,"(0_Marital, 0_Occ)",2
65,0.077145,"(1_PC1, 0_Occ)",2
66,0.069738,"(5_PC1, 0_Occ)",2
67,0.054020,"(8_PC1, 0_Occ)",2
...,...,...,...
941,0.259079,"(M, 26-35)",2
942,0.055646,"(36-45, F)",2
943,0.140741,"(M, 36-45)",2
944,0.059621,"(M, 46-50)",2


In [ ]:
#Para k=3

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets3=frequent_itemsets[frequent_itemsets['length'] == 3]
frequent_itemsets3

,support,itemsets,length
946,0.053839,"(C, 0_PC3, 0_Occ)",3
947,0.051852,"(0_PC2, 0_Marital, 0_Occ)",3
948,0.064318,"(0_Marital, 0_PC3, 0_Occ)",3
949,0.051671,"(1_PC1, 5_PC1, 0_Occ)",3
950,0.059440,"(1_PC1, 0_PC2, 0_Occ)",3
...,...,...,...
5863,0.100813,"(15_PC3, M, 26-35)",3
5864,0.081301,"(M, 16_PC3, 17_PC3)",3
5865,0.055465,"(M, 16_PC3, 18-25)",3
5866,0.106956,"(M, 16_PC3, 26-35)",3


In [ ]:
#Reglas de asociación
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(0_Occ),(0_PC3),0.118338,0.939115,0.108762,0.919084,0.978671,-0.002370,0.752450
1,(1_Occ),(0_PC3),0.089431,0.939115,0.083107,0.929293,0.989541,-0.000878,0.861092
2,(4_Occ),(0_PC3),0.125745,0.939115,0.117615,0.935345,0.995986,-0.000474,0.941692
3,(7_Occ),(0_PC3),0.113640,0.939115,0.107498,0.945946,1.007274,0.000776,1.126378
4,(7_Occ),(M),0.113640,0.720867,0.091057,0.801272,1.111539,0.009137,1.404596
...,...,...,...,...,...,...,...,...,...
3167,"(16_PC3, 9_PC3)",(M),0.083650,0.720867,0.067209,0.803456,1.114568,0.006908,1.420203
3168,"(15_PC3, 14_PC3)",(M),0.104968,0.720867,0.084553,0.805508,1.117415,0.008885,1.435187
3169,"(15_PC3, 16_PC3)",(M),0.158988,0.720867,0.131346,0.826136,1.146031,0.016737,1.605470
3170,"(15_PC3, 17_PC3)",(M),0.088528,0.720867,0.073532,0.830612,1.152240,0.009715,1.647892


Pero tenemos que quitar aquellas reglas que tengan un 0 en el antecedente o en la consecuencia. 

In [ ]:
mask = (rules['antecedents'].apply(lambda x: '0_PC2' in x) | rules['consequents'].apply(lambda x: '0_PC2' in x) | rules['consequents'].apply(lambda x: '0_PC3' in x) | rules['consequents'].apply(lambda x: '0_PC3' in x))


rules_with_zero = rules[mask]

In [ ]:
rules = rules.drop(rules_with_zero.index)

In [ ]:
#vemos reglas de alta confianza
rulesaltaconfianza = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.95)
rulesaltaconfianza

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(A),(0_PC3),0.181933,0.939115,0.173442,0.953327,1.015133,0.002586,1.304499
1,(B),(0_PC3),0.296658,0.939115,0.285637,0.962850,1.025274,0.007041,1.638912
2,(2_PC2),(1_PC1),0.440650,0.672267,0.440650,1.000000,1.487503,0.144416,inf
3,(6_PC3),(1_PC1),0.072629,0.672267,0.072629,1.000000,1.487503,0.023803,inf
4,(15_PC3),(1_PC1),0.290515,0.672267,0.278771,0.959577,1.427374,0.083468,8.107601
...,...,...,...,...,...,...,...,...,...
938,"(26-35, 17_PC3)",(0_PC3),0.077868,0.939115,0.074255,0.953596,1.015420,0.001128,1.312078
939,"(F, 17_PC3)",(0_PC3),0.058537,0.939115,0.055646,0.950617,1.012248,0.000673,1.232927
940,"(M, 17_PC3)",(0_PC3),0.161156,0.939115,0.154291,0.957399,1.019470,0.002947,1.429202
941,"(M, 18_PC3)",(0_PC3),0.060163,0.939115,0.058537,0.972973,1.036053,0.002037,2.252755


In [65]:
for i in range(len(rulesaltaconfianza)):
  if 'B' in rulesaltaconfianza['antecedents'][i]:
    print(rulesaltaconfianza.iloc[i])

antecedents                (B)
consequents            (0_PC3)
antecedent support    0.296658
consequent support    0.939115
support               0.285637
confidence             0.96285
lift                  1.025274
leverage              0.007041
conviction            1.638912
Name: 1, dtype: object
antecedents           (B, 2_Stay)
consequents               (0_PC3)
antecedent support       0.106052
consequent support       0.939115
support                  0.103162
confidence               0.972743
lift                     1.035808
leverage                 0.003566
conviction               2.233729
Name: 86, dtype: object
antecedents           (B, 0_Marital)
consequents                  (0_PC3)
antecedent support          0.173261
consequent support          0.939115
support                     0.167118
confidence                  0.964546
lift                         1.02708
leverage                    0.004406
conviction                  1.717323
Name: 87, dtype: object
antecedents

In [69]:
for i in range(len(rulesaltaconfianza)):
  if rulesaltaconfianza['confidence'][i] > 0.98:
    print(rulesaltaconfianza.iloc[i])

Se han truncado las últimas 5000 líneas del flujo de salida.
antecedents           (2_PC2, 0_Marital)
consequents                      (1_PC1)
antecedent support              0.263415
consequent support              0.672267
support                         0.263415
confidence                           1.0
lift                            1.487503
leverage                         0.08633
conviction                           inf
Name: 203, dtype: object
antecedents           (0_Marital, 5_PC1)
consequents                      (0_PC3)
antecedent support               0.33514
consequent support              0.939115
support                         0.331165
confidence                       0.98814
lift                            1.052204
leverage                         0.01643
conviction                      5.133736
Name: 208, dtype: object
antecedents           (6_PC1, 0_Marital)
consequents                      (0_PC3)
antecedent support              0.141644
consequent support          

NO BORRAR: 

In [ ]:
#frequent_itemsets = apriori(data, min_support=0.6, use_colnames=True)
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
#frequent_itemsets1=frequent_itemsets[frequent_itemsets['length'] == 1]

In [ ]:
#frequent_itemsets[ (frequent_itemsets['length'] >1 ) & #esto sería para k=2
                   #(frequent_itemsets['support'] >= 0.8) ]

Vale, hay un apartado que pide representar gráficamente las reglas: 

In [ ]:
plot_confidence_vs_support(rulesaltaconfianza)

NameError: ignored

##Evaluación

##Despliegue